In [1]:
import sys
import os

# If running as script, adjust sys.path for sibling folder access
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../../..'))
data_folder = os.path.join(project_root, "Data", "CleanedText")
chroma_db_path = os.path.join(project_root, "Chroma_db_databases")
chromadb_sqlite_path = os.path.join(chroma_db_path, "chroma.sqlite3")

# Import libraries
import ollama
import chromadb

# Connect to ChromaDB (PersistentClient with full DB folder path)
client = chromadb.PersistentClient(path=chroma_db_path)
# Your main collection name (change as necessary)
collection = client.get_collection("policy_docs")

def rag_answer(user_query, top_k=5):
    # 1. Embed the user question
    query_embedding = ollama.embeddings(
        model="nomic-embed-text:latest",
        prompt=user_query
    )["embedding"]

    # 2. Fetch the most relevant docs
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["documents", "metadatas"]
    )
    context_docs = results["documents"][0]
    sources = results["metadatas"][0]

    # 3. Build the LLM context
    context = "\n\n".join([
        f"Source: {sources[i]['source']}\nContent: {doc[:800]}"
        for i, doc in enumerate(context_docs)
    ])

    # 4. Prompt for Qwen2.5 LLM
    prompt = (
        "Given the following regulatory sources and clauses, answer the question below as accurately as possible. "
        "Cite the document source in your answer.\n"
        "==== Context ====\n"
        f"{context}\n"
        "==== End Context ====\n\n"
        f"Question: {user_query}\nAnswer:"
    )

    # 5. Generate LLM answer
    response = ollama.generate(model="qwen2.5:7b", prompt=prompt)["response"]
    return response

# --- Example usage ---
if __name__ == "__main__":
    question = "What are the mandatory anti-ragging committees according to UGC regulations?"
    answer = rag_answer(question)
    print("\nGenerated Answer:\n", answer)


NotFoundError: Collection [policy_docs] does not exists